In [1]:
# the following Dhondt algorithm based on idea from https://gist.github.com/brunosan/96288a8612894fca718aacbcc501ee09
# but it implemented using Keras and TensorFlow
import tensorflow as tf
import  tensorflow.keras.backend as K
import  tensorflow.keras as keras
from tensorflow.keras.layers import Input
#
from tensorflow.keras.models import Model
#
# UNCOMMENT the following line if you want to save the model and use it in BigQuery
#@tf.keras.utils.register_keras_serializable()
class DHondt_TensorFlow_KERAS_1D(keras.layers.Layer):                       
    def __init__(self,  **kwargs):
        super(DHondt_TensorFlow_KERAS_1D, self).__init__(**kwargs)           
        
    # about get_config  see the following link:
    # https://towardsdatascience.com/how-to-write-a-custom-keras-model-so-that-it-can-be-deployed-for-serving-7d81ace4a1f8
    def get_config(self):
        config = super().get_config()
        # save constructor args
        #config['input_dim'] = self.input_dim
        # 
        return config
    @tf.function
    def call(self, inputs):
        inputs = tf.transpose(inputs)
        inputs = tf.squeeze(inputs) 
        #
        ##########################################################################
        #
        seats = tf.cast(tf.math.floor(inputs), dtype = tf.float32)
        #
        nSeats = tf.math.reduce_sum(inputs)
        print("nSeats: ", nSeats)
        #
        if tf.greater(nSeats, 0):
           votes = tf.divide(inputs, nSeats) 
           #
           nSeats = tf.cast(nSeats, dtype = tf.int32)
           #
           allocated_seat_number = tf.cast(tf.math.reduce_sum(seats), dtype = tf.int32)
           #
           ##########################################################################
           #
           t_votes = tf.identity(votes)
           #
           while tf.math.less(allocated_seat_number, nSeats):
                    #
                    next_seat = tf.math.argmax(t_votes)
                    #
                    ############################################################
                    #
                    update = tf.gather(seats,  next_seat)
                    #
                    update = tf.add(update, 1.0)
                    #
                    update = tf.convert_to_tensor([update])
                    #
                    seats = tf.tensor_scatter_nd_update(seats, [[next_seat]], update)
                    #
                    allocated_seat_number = tf.add(allocated_seat_number, 1)
                    #
                    ############################################################
                    #
                    denumerator = tf.add(update, 1.0)
                    #
                    numerator   = tf.gather(votes,  next_seat)
                    #
                    update      = tf.divide(numerator, denumerator)
                    #
                    t_votes     = tf.tensor_scatter_nd_update(t_votes,  [[next_seat]], update)
           ##########################################################################
           seats = tf.reshape(seats, [tf.size(seats), 1])
        #
        else:
           seats = tf.zeros([tf.size(inputs), 1]) 
        return  seats             
#
#
input_layer  = Input(shape = (1))
#
output_layer =  DHondt_TensorFlow_KERAS_1D()(input_layer)  ## TESTING
#
model = Model(inputs = input_layer, outputs = output_layer)
#
total_seats = 10
#
inputs = tf.constant([[0.03], [0.07], [0.04], [0.06], [0.5], [0.03], [0.07], [0.04], [0.06], [0.05], [0.05]])
inputs = tf.math.multiply(inputs, total_seats)
print(inputs)
#
seats = model.predict(inputs)
print("prediction, seats:")
print(seats)
model.save("DHondt_TensorFlow_KERAS_1D")      

nSeats:  Tensor("Sum:0", shape=(), dtype=float32)
tf.Tensor(
[[0.29999998]
 [0.7       ]
 [0.39999998]
 [0.59999996]
 [5.        ]
 [0.29999998]
 [0.7       ]
 [0.39999998]
 [0.59999996]
 [0.5       ]
 [0.5       ]], shape=(11, 1), dtype=float32)
1/1 [==============================] - 0s 124ms/step
prediction, seats:
[[0.]
 [1.]
 [0.]
 [0.]
 [8.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]]
nSeats:  Tensor("Sum:0", shape=(), dtype=float32)
nSeats:  Tensor("Sum:0", shape=(), dtype=float32)
nSeats:  Tensor("Sum:0", shape=(), dtype=float32)
nSeats:  Tensor("Sum:0", shape=(), dtype=float32)
INFO:tensorflow:Assets written to: DHondt_TensorFlow_KERAS_1D\assets
